In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
import os
import matplotlib.pyplot as plt
import pickle
from tqdm import tqdm 
from datetime import date 
from datetime import datetime
from collections import defaultdict 

In [2]:
# from foodparser import foodparser

In [3]:
# fp = foodparser.FoodParser()

In [4]:
# !pip install pyspellchecker

%load_ext autoreload
%autoreload 2

# %run '/content/drive/My Drive/Colab Data/public_app_analysis/utils/publicAnalysisHelper.py'
# %run ../utils/publicAnalysisHelper.py
# %run ../food_parser/food_parser.py

In [5]:
# from google.colab import drive
# drive.mount('/content/drive')
file_dir = '../'

data_dir = file_dir + 'test_data/'
result_dir = data_dir + 'results/'

In [6]:
import sys
sys.path.insert(1, '../foodparser/')

from foodparser import *

[nltk_data] Downloading package words to /Users/joey-hou/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/joey-
[nltk_data]     hou/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/joey-
[nltk_data]     hou/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/joey-hou/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
fp = FoodParser()
fp.initialization()

fp.parse_food('Espresso', return_sentence_tag = True)

(['espresso'], 1, 0, [])

In [18]:
# fp = FoodParser()

# fp.all_gram_set[0].add('yoghurt')

'yoghurt' in fp.all_gram_set[0]

True

# Merge combine_df files

In [8]:
# gram_set = pd.read_csv('../food_parser/final_database/gram_set.csv')
# gram_tag = pd.read_csv('../food_parser/final_database/gram_tag.csv')
# gram_set.shape, gram_tag.shape

In [9]:
# gram_set.drop(columns = ['gram_type']).merge(gram_tag, on = 'gram_key', how = 'outer')\
#     .to_csv('../food_parser/final_database/combined_gram_set.csv', index = False)

In [10]:
# gram_tag.drop(columns = ['gram_type']).merge(gram_set, on = 'gram_key')

# Load a sample of data

In [11]:
firefighter_data_pickle_file = open(data_dir + 'raw/FFFoodData_Unique_Time_All_Aug2020.pickle', 'rb')
firefighter_data = pickle.load(firefighter_data_pickle_file).reset_index(drop = True)
firefighter_data = firefighter_data.rename(columns = {'PID': 'unique_code'})
firefighter_data.head(2)

,unique_code,caloric,day,desc_text,food_type,local_time,original_logtime,original_logtime_notz,participant,study,time,tz
0,9bbi18050004,1,2018-07-18,kind bar && coffee,f,9.583333,07-18-2018 09:35AM PDT,2018-07-18 09:35:00,Hero-001,TRE,09:35:00,PDT
1,9bbi18050004,1,2018-06-24,potatoe,f,19.116667,06-24-2018 07:07PM PDT,2018-06-24 19:07:00,Hero-001,TRE,19:07:00,PDT


In [16]:
firefighter_data.loc[firefighter_data.desc_text.str.contains(',')]

,unique_code,caloric,day,desc_text,food_type,local_time,original_logtime,original_logtime_notz,participant,study,time,tz
16,9bbi18050004,1,2018-10-15,"yogurt, kind bar",f,12.650000,10-15-2018 12:39PM PDT,2018-10-15 12:39:00,Hero-001,TRE,12:39:00,PDT
24,9bbi18050004,1,2018-08-28,"yogurt, banana, kind bar",f,9.250000,08-28-2018 09:15AM PDT,2018-08-28 09:15:00,Hero-001,TRE,09:15:00,PDT
30,9bbi18050004,1,2019-02-02,"yogurt, yogurt berries",f,10.200000,02-02-2019 10:12AM PST,2019-02-02 10:12:00,Hero-001,TRE,10:12:00,PST
46,9bbi18050004,1,2018-09-19,"2 cookies, eggs sausage toast && coffee",f,9.316667,09-19-2018 09:19AM PDT,2018-09-19 09:19:00,Hero-001,TRE,09:19:00,PDT
64,9bbi18050004,1,2018-09-05,"donut and coffee, eggcheese avo burrito, kind bar",f,14.000000,09-05-2018 02:00PM PDT,2018-09-05 14:00:00,Hero-001,TRE,14:00:00,PDT
...,...,...,...,...,...,...,...,...,...,...,...,...
109607,9bbi19120173,1,2020-05-01,"pre work out 3 scoops, protein shake with mix ...",b,13.033333,05-01-2020 01:02PM PDT,2020-05-01 13:02:00,Hero-153,TRE,13:02:00,PDT
109632,9bbi19120173,0,2020-01-12,"sparking water, spark",b,18.100000,01-12-2020 06:06PM PST,2020-01-12 18:06:00,Hero-153,TRE,18:06:00,PST
109679,9bbi18100060,1,2018-11-29,"tomato soup, turkey grilled cheese",f,12.116667,11-29-2018 12:07:00 PDT,2018-11-29 12:07:00,Hero-048,SOC,12:07:00,PDT
109681,9bbi18100060,1,2018-12-02,"pb bagel, mixed nuts",f,20.750000,12-02-2018 20:45:00 PDT,2018-12-02 20:45:00,Hero-048,SOC,20:45:00,PDT


In [12]:
firefighter_data.shape

(109708, 12)

In [17]:
firefighter_data_expanded = fp.expand_entries(firefighter_data)

100%|██████████| 109708/109708 [00:31<00:00, 3512.49it/s]


In [19]:
sample_df = firefighter_data_expanded
if 'ID' not in sample_df:
    sample_df['ID'] = range(sample_df.shape[0])

total_unknown = 0
total_token = 0
parsed = []
total_unknown_tokens = []
for i in tqdm(range(sample_df.shape[0])):
    entry = sample_df.iloc[i].desc_text
    result, num_token, num_unknown, unknown_tokens = fp.parse_food(entry, return_sentence_tag = True)
    result = [(f, fp.find_food_type(f)) for f in result]
    parsed.append(result)
    total_token += num_token
    total_unknown += num_unknown
    total_unknown_tokens += unknown_tokens

100%|██████████| 162568/162568 [01:19<00:00, 2038.31it/s]


- parsed

In [20]:
sample_df['parsed'] = parsed

# Build search functions

## Look-up-dict

In [97]:
required_columns = ['gram_key', 'food_type', 'gram_type']
tag_lst = ['tag1', 'tag2', 'tag3']
gram_tag_df = fp.read_gram_set().fillna('empty')[required_columns + tag_lst]
gram_tag_df.head()

,gram_key,food_type,gram_type,tag1,tag2,tag3
0,vodka,b,1,empty,Alcohol,empty
1,monster,b,1,empty,Caffeine,empty
2,coke,b,1,empty,Caffeine,empty
3,mocha,b,1,coffee,Caffeine,High-sugar
4,milk,b,1,empty,Dairy,protein


In [98]:
def basic_clean(s):
    tmp_s = s.lower().replace('?', '')
    if len(tmp_s) == 0:
        return 'empty'
    else:
        return tmp_s
gram_tag_df.tag1 = gram_tag_df.tag1.apply(basic_clean)
gram_tag_df.tag2 = gram_tag_df.tag2.apply(basic_clean)
gram_tag_df.tag3 = gram_tag_df.tag3.apply(basic_clean)

In [99]:
gram_tag_df.head()

,gram_key,food_type,gram_type,tag1,tag2,tag3
0,vodka,b,1,empty,alcohol,empty
1,monster,b,1,empty,caffeine,empty
2,coke,b,1,empty,caffeine,empty
3,mocha,b,1,coffee,caffeine,high-sugar
4,milk,b,1,empty,dairy,protein


In [100]:
for row_idx in range(gram_tag_df.shape[0]):
    curr_row = gram_tag_df.iloc[row_idx]
    if 'coffee' in curr_row.gram_key:
        if 'coffee' not in [curr_row['tag'+str(i)] for i in [1,2,3]] and \
           'caffeine' not in [curr_row['tag'+str(i)] for i in [1,2,3]]:
            print(curr_row.gram_key)

decaf coffee milk


In [101]:
look_up_dict = {}
for row_i in range(gram_tag_df.shape[0]):
    curr_row = gram_tag_df.iloc[row_i]
    tmp_lst = [curr_row['gram_key']]
    for tag in tag_lst:
        item = curr_row[tag]
        if item == 'empty':
            continue
        tmp_lst.append(item)
    look_up_dict[tmp_lst[0]] = tmp_lst

In [102]:
gram_tag_df.iloc[0].values

array(['vodka', 'b', 1.0, 'empty', 'alcohol', 'empty'], dtype=object)

In [103]:
[i for i in list(look_up_dict.items())[:3]]

[('vodka', ['vodka', 'alcohol']),
 ('monster', ['monster', 'caffeine']),
 ('coke', ['coke', 'caffeine'])]

## One-hot-dict

In [104]:
counter = 0
idx_to_tag = {}
tag_to_idx = {}
for key in look_up_dict.keys():
    for tag in look_up_dict[key]:
        if tag not in tag_to_idx.keys():
            idx_to_tag[counter] = tag
            tag_to_idx[tag] = counter
            counter += 1

In [105]:
all_tag_arr = np.array(list(tag_to_idx.keys()))

In [106]:
parsed[:3] # This is the array we need to analysis

[[('kind bar', 'f'), ('coffee', 'b')],
 [('potatoe', 'f')],
 [('egg', 'f'), ('sausage', 'f')]]

In [107]:
vec_size = counter
def encode_single_item(item):
    tags = look_up_dict[item]
    return [tag_to_idx[t] for t in tags]

def encode_single_parsed_entry(result_lst):
    encoded_arr = np.array([0] * vec_size)
    for pair in result_lst:
        item = pair[0]
        encoded_arr[encode_single_item(item)] = 1
    return encoded_arr

def decode_item_arr(arr):
    return all_tag_arr[arr == 1]

def find_str_contain(df, s):
    return df.loc[df.desc_text.apply(lambda x: s in x)].reset_index(drop = True)

In [108]:
vec_size

2015

In [109]:
parsed_encoded_arrays = []
parsed_tagged_text = []
for p in parsed:
    encoded_arr = encode_single_parsed_entry(p)
    parsed_encoded_arrays.append(encoded_arr)
    parsed_tagged_text.append(decode_item_arr(encoded_arr))

parsed_encoded_arrays = np.array(parsed_encoded_arrays)

In [110]:
parsed_tagged_text[1630:1635]

[array(['meat', 'beef', 'dressing', 'steak', 'fat free'], dtype='<U47'),
 array(['meat', 'beef', 'dressing', 'steak', 'fat free'], dtype='<U47'),
 array(['coffee', 'dairy', 'latte', 'vanilla', 'iced'], dtype='<U47'),
 array(['wheat', 'individual', 'pancake'], dtype='<U47'),
 array(['wheat', 'individual', 'pancake'], dtype='<U47')]

In [111]:
# encode_single_item('fish taco')

In [112]:
parsed_encoded_arrays[:2]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [113]:
parsed_encoded_arrays.shape

(162568, 2015)

In [114]:
# parsed_encoded_arrays[:, 1].shape

In [74]:
# sample_df.shape

In [75]:
# len(parsed)

In [115]:
def search(encoded_arr, tag, id_lst):
    idx = tag_to_idx[tag]
    selected_entry_spots = id_lst[encoded_arr[:, idx] != 0]
    return selected_entry_spots

def backward_search(tag):    
    return [k for k in look_up_dict.keys() if tag in look_up_dict[k]]

In [116]:
# backward_search('caffeine')

## Deploy on Coffee example

In [117]:
coffee_found = \
    list(search(parsed_encoded_arrays, 'coffee', np.array(range(162568)))) + \
    list(search(parsed_encoded_arrays, 'caffeine', np.array(range(162568))))
coffee_found = list(set(coffee_found))
coffee_spot = sample_df.reset_index(drop = True).loc[list(coffee_found)].reset_index(drop = True)

In [118]:
coffee_spot.head(2)

,unique_code,caloric,day,desc_text,food_type,local_time,original_logtime,original_logtime_notz,participant,study,time,tz,ID,parsed
0,9bbi18050004,1,2018-07-18,kind bar && coffee,f,9.583333,07-18-2018 09:35AM PDT,2018-07-18 09:35:00,Hero-001,TRE,09:35:00,PDT,0,"[(kind bar, f), (coffee, b)]"
1,9bbi18050004,1,2018-06-07,banana && coffee,f,9.050000,06-07-2018 09:03AM PDT,2018-06-07 09:03:00,Hero-001,TRE,09:03:00,PDT,10,"[(banana, f), (coffee, b)]"


In [119]:
len(coffee_found)

14547

In [120]:
coffee_spot.shape

(14547, 14)

In [121]:
find_str_contain(coffee_spot, 'coffee').shape

(10725, 14)

In [122]:
# for k in list(look_up_dict.keys()):
#     if 'coffee' in look_up_dict[k]:
#         print(look_up_dict[k])

In [124]:
# find_str_contain(sample_df, 'latte')

In [72]:
look_up_dict['cappuccino']

['cappuccino', 'coffee', 'coffee']

In [40]:
coffee_spot.head(2)

,PID,caloric,day,desc_text,food_type,local_time,original_logtime,original_logtime_notz,participant,study,time,tz,ID,parsed
0,9bbi18050004,1,2018-07-18,kind bar && coffee,f,9.583333,07-18-2018 09:35AM PDT,2018-07-18 09:35:00,Hero-001,TRE,09:35:00,PDT,0,"[(kind bar, f), (coffee, b)]"
1,9bbi18050004,1,2018-06-07,banana && coffee,f,9.050000,06-07-2018 09:03AM PDT,2018-06-07 09:03:00,Hero-001,TRE,09:03:00,PDT,10,"[(banana, f), (coffee, b)]"


In [125]:
coffee_spot.to_csv(result_dir + 'coffee_events.csv', index = False)

In [ ]:
food_dict = defaultdict(lambda : 0)
food_type_dict = defaultdict(lambda : 0)

coffee_backsearch_result = backward_search('coffee')


for i in range(coffee_spot.shape[0]):
    food_lst = coffee_spot.parsed.values[i]
    for pair in food_lst:
        if pair[0] not in coffee_backsearch_result:
            food_dict[pair[0]] += 1
            food_type_dict[pair[1]] += 1

In [ ]:
food_count_lst = []
for pair in dict(food_dict).items():
    food_count_lst.append({
        'food': pair[0],
        'count': pair[1]
    })
food_count_df = pd.DataFrame(food_count_lst).sort_values(by = 'count', ascending = False).reset_index(drop = True)
food_count_df

## Deploy on alcohol example

In [127]:
alcohol_found = list(search(parsed_encoded_arrays, 'alcohol', np.array(range(len(parsed_encoded_arrays)))))
alcohol_found = list(set(alcohol_found))
alcohol_spot = sample_df.reset_index(drop = True).loc[list(alcohol_found)].reset_index(drop = True)

In [128]:
alcohol_spot.shape

(6636, 14)

In [130]:
# alcohol_spot

In [131]:
alcohol_spot.to_csv(result_dir + 'alcohol_events.csv', index = False)

In [44]:
# food_count_df.to_csv('food_count_df.csv', index = False)

In [45]:
# sample_df.to_csv('sample_df.csv', index = False)